In [ ]:
##see later
import re

def add_spaces_around_punctuation(text):
    # Define a regular expression pattern to match punctuation marks without surrounding spaces
    pattern = r'(?<=[^\s\d])(?=[.,;:?!])|(?<=[.,;:?!])(?=[^\s])'

    # Add spaces around punctuation marks
    modified_text = re.sub(pattern, ' ', text)

    return modified_text

def add_spaces_around_parentheses(text):
    # Define a regular expression pattern to match parentheses without surrounding spaces
    pattern = r'(?<=\w)(\()|(\))(?=\w)'

    # Add spaces around parentheses
    modified_text = re.sub(pattern, ' \\1\\2 ', text)

    return modified_text

# Example sentences
sentences = [
    "Kaur,",
    "28.07.1988",
    "(pf chang)"
]

# Apply both transformations to each sentence
modified_sentences = [add_spaces_around_punctuation(add_spaces_around_parentheses(sentence)) for sentence in sentences]

# Print modified sentences
for original, modified in zip(sentences, modified_sentences):
    print(f"Original: {original}")
    print(f"Modified: {modified}")
    print()


In [186]:
import json
with open("NER/NER_TRAIN_JUDGEMENT.json", "r") as f:
    data = json.load(f)

with open("NER/NER_TEST_JUDGEMENT.json","r") as f:
    data_test = json.load(f)

In [187]:
import numpy as np
#SPlitting
from skmultilearn.model_selection import iterative_train_test_split

In [188]:
labels = {}
for i in range(len(data)):
    for j in range(len(data[i]["annotations"][0]["result"])):
        label_text = data[i]["annotations"][0]["result"][j]["value"]["text"]
        if(label_text in labels):
            continue
        else:
            labels[label_text] = data[i]["annotations"][0]["result"][j]["value"]["labels"][0]

unique_labels = set()
for text,keys in labels.items():
    unique_labels.add(keys)

indices = {key:value for key,value in zip(unique_labels,range(13))}

In [189]:
X = np.arange(len(data))
X = X.reshape(-1, 1)
Y = np.zeros((len(data),13))

In [190]:
for i in range(len(data)):
    for j in range(len(data[i]["annotations"][0]["result"])):
            Y[i][indices[data[i]["annotations"][0]["result"][j]["value"]["labels"][0]]] += 1

In [191]:
X_train, y_train, X_val, y_val = iterative_train_test_split(X, Y, test_size = 0.15)

In [192]:
data_train = [data[i[0]] for i in X_train]
data_val = [data[i[0]] for i in X_val]

In [193]:
text_train = []
case_ids_train = []
text_val = []
case_ids_val = []
text_test = []
case_ids_test = []
for i in range(len(data_train)):
    text_train.append(data_train[i]["data"]["text"])
    case_ids_train.append(data_train[i]["id"])

for i in range(len(data_val)):
    text_val.append(data_val[i]["data"]["text"])
    case_ids_val.append(data_val[i]["id"])

for i in range(len(data_test)):
    text_test.append(data_test[i]["data"]["text"])
    case_ids_test.append(data_test[i]["id"])

In [194]:
import re
def bio_label_partI(text,annotations):
    tokens = text.split()
    # print(tokens)
    bio_temp = ["O" for _ in range(len(tokens))]
    for annotation in annotations:
        for result in annotation['result']:
            start = result['value']['start']
            end = result['value']['end']
            entity_text = result['value']['text']
            labels = result['value']['labels'][0]
            
            entities = entity_text.split()            

            start_index = 0
            curr_pos = 0
            for i, token in enumerate(tokens):
                curr_pos = text.find(token, curr_pos)
                if curr_pos >= start:
                    start_index = i
                    break
                curr_pos += len(token)

            end_index = start_index+len(entities)-1           
            try:
                bio_temp[start_index] = "B_"+labels
                for i in range(start_index+1,end_index+1):
                    bio_temp[i] = "I_" + labels
            except IndexError:
                continue
    return bio_temp    

In [195]:
BIO_labels_train = []
BIO_labels_val = []
BIO_labels_test = []

In [196]:
for i,line in enumerate(text_train):
    BIO_labels_train.append(bio_label_partI(line,data_train[i]["annotations"]))

In [197]:
for i,line in enumerate(text_test):
    BIO_labels_test.append(bio_label_partI(line,data_test[i]["annotations"]))

In [198]:
for i,line in enumerate(text_val):
    BIO_labels_val.append(bio_label_partI(line,data_val[i]["annotations"]))

In [199]:
data = {}

# Iterate over IDs, texts, and labels simultaneously
for id_, text, label in zip(case_ids_test, text_test, BIO_labels_test):
    data[id_] = {
        "text": text,
        "labels": label
    }

# Convert data to JSON string
json_string = json.dumps(data, indent=4)
with open("dataset_1_test.json", "w") as json_file:
    json_file.write(json_string)

In [200]:
data = {}

# Iterate over IDs, texts, and labels simultaneously
for id_, text, label in zip(case_ids_train, text_train, BIO_labels_train):
    data[id_] = {
        "text": text,
        "labels": label
    }

# Convert data to JSON string
json_string = json.dumps(data, indent=4)
with open("dataset_1_train.json", "w") as json_file:
    json_file.write(json_string)

In [201]:
data = {}

# Iterate over IDs, texts, and labels simultaneously
for id_, text, label in zip(case_ids_val, text_val, BIO_labels_val):
    data[id_] = {
        "text": text,
        "labels": label
    }

# Convert data to JSON string
json_string = json.dumps(data, indent=4)
with open("dataset_1_val.json", "w") as json_file:
    json_file.write(json_string)